<a href="https://colab.research.google.com/github/njuerect/TF2.0-Keras-Tutorial/blob/master/Keras_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, GlobalMaxPool2D
from tensorflow.keras import Model
from tensorflow.keras.layers import Reshape, Conv2DTranspose, UpSampling2D

In [7]:
num_words = 2000
num_tags = 12
num_departments = 4

body_input = keras.Input(shape=(None,), name='body')
title_input = keras.Input(shape=(None,), name='title')
tag_input = keras.Input(shape=(num_tags,), name='tag')

body_feat = layers.Embedding(num_words, 64)(body_input)
title_feat = layers.Embedding(num_words, 64)(title_input)

body_feat = layers.LSTM(32)(body_feat)
title_feat = layers.LSTM(128)(title_feat)
features = layers.concatenate([title_feat, body_feat, tag_input])

priority_pred = layers.Dense(1, activation='sigmoid', name='priority')(features)
department_pred = layers.Dense(num_departments, activation='softmax', name='department')(features)

model = Model(inputs = [body_input, title_input, tag_input],
             outputs = [priority_pred, department_pred])
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
body (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 64)     128000      title[0][0]                      
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 64)     128000      body[0][0]                       
______________________________________________________________________________________________

In [0]:
model.compile(optimizer = keras.optimizers.RMSprop(1e-3), 
             loss = {'priority': 'binary_crossentropy',
                    'department': 'categorical_crossentropy'},
             loss_weights=[1., 0.2])

In [9]:
import numpy as np
# 载入输入数据
title_data = np.random.randint(num_words, size=(1280, 10))
body_data = np.random.randint(num_words, size=(1280, 100))
tag_data = np.random.randint(2, size=(1280, num_tags)).astype('float32')
# 标签
priority_label = np.random.random(size=(1280, 1))
department_label = np.random.randint(2, size=(1280, num_departments))
# 训练
history = model.fit(
    {'title': title_data, 'body':body_data, 'tag':tag_data},
    {'priority':priority_label, 'department':department_label},
    batch_size=32,
    epochs=5
)

Epoch 1/5
1280/1280 [==============================] - 10s 8ms/sample - loss: 1.2570 - priority_loss: 0.6953 - department_loss: 2.8087
Epoch 2/5
1280/1280 [==============================] - 10s 8ms/sample - loss: 1.2089 - priority_loss: 0.6509 - department_loss: 2.7899
Epoch 3/5
1280/1280 [==============================] - 10s 8ms/sample - loss: 1.1350 - priority_loss: 0.5798 - department_loss: 2.7762
Epoch 4/5
1280/1280 [==============================] - 10s 8ms/sample - loss: 1.0915 - priority_loss: 0.5400 - department_loss: 2.7571
Epoch 5/5
1280/1280 [==============================] - 10s 8ms/sample - loss: 1.0669 - priority_loss: 0.5228 - department_loss: 2.7203
